# Portfolio Management System

---
Name: Arvin Corotana

Student Id: 20233408

Course Type: Taught Masters 

---



### Integrity statement

I, Arvin Corotana, have read and understood the School's Academic Integrity Policy, as well as guidance relating to this module, and confirm that this submission complies with the policy. The content of this file is my own original work, with any significant material copied or adapted from other sources clearly indicated and attributed.

## Imports




In [17]:

from datetime import datetime, timedelta #https://docs.python.org/3/library/datetime.html
import yfinance as yf   #https://github.com/ranaroussi/yfinance
import matplotlib.pyplot as plt #https://matplotlib.org/stable/api/pyplot_summary.html
import numpy as np #https://numpy.org/doc/
import pandas as pd #https://pandas.pydata.org/pandas-docs/stable/
from scipy.optimize import minimize #https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html
from apscheduler.schedulers.blocking import BlockingScheduler #https://apscheduler.readthedocs.io/en/stable/
from email.message import EmailMessage #https://docs.python.org/3/library/email.message.html
import ssl #https://docs.python.org/3/library/ssl.html
import smtplib #https://docs.python.org/3/library/smtplib.html
from time import gmtime #https://docs.python.org/3/library/time.html#time.gmtime
import mplfinance as mpf #https://github.com/matplotlib/mplfinance


## Discussion of project



### Aim of project
The main aim of the project is to meet the needs of somebody who is not sure about how they should go about investing. The main aim is to allow the user to create portfolios and view their performance to gain familiarity with the fluctuations and get a suggestion on how they should invest their money in their portfolio. The secondary aim is to create an alert system to advise the user what to do when there is a pattern found in the stock data.
### Brief overview of how code works
There are 3 main sections of code. The first being the optimal portfolio class where portfolios can be created, edited, viewed and selected leading to the optimisation of the portfolio given constraints. The code also includes a way to visualise the selected portfolio stocks. There is the alert system code that takes an instance from the selection of a portfolio and runs the alert system with that portfolio where the performance of the performance is compared to the previous days open and close prices. This results in alerts at a certain time giving updates on the stocks performance recently. There are also main menu functions which just give the user a way of naviagting the different tools with different functions called with diferent options selected.
### Design decisions made in structuring of the code
Structured the code into 3 main sections with a portfolio class so it can be run multiple times with attributes for different instances. Then for the alert system and main menu functions were created as classes are not needed.
### How you would improve / develop the code if given more time
Discussed within individual sections. However, an overall improvement that could have been made is the debugging process. I decided to put print statements in between lines of code to determine where the flow may stop and works. I also just made sure to test different inputs along with the same inputs to make sure the output is the same. This is fine for me but maybe using some sort of test system may be beneficial.

The alert system does not work as intended as running it after selecting whihc portfolio the tracking starts but I am not sure how to come back to the main menu as it will run forever and run te function at 9am. Therefore all other functions run as intended but the alter system portfolio selection leads to the stopping of the program. 

## Optimal Portfolio Class

Overall, there are functions that contribute to the creation, editing, viewing and optimisation of portfolios. The functions interact with eachother and called in the main menu functions creating a flow. The way the code worksis explained in the comments. Much of the structure of the calculations for the optimization was heavily inspired from Ryan O'Connell and his youtube video, the 1st reference. If given more time I would have liked to implement different and more complex algorithms with the user getting suggestions from them with an explanation of the algorithms pros and cons so they have a broader view on the potential of their portfolio. A check for if the stock symbol entered is valid would be an improvement.

In [18]:

#Create a class so that information can be passed between functions easier
class PortfolioOptimizer:
    
    def __init__(self):
        self.portfolios = {} #initialise dictionary for the portfolios and their stocks
        self.all_stocks=[]
        

#Need to get the users input when creating portfolios and can then run the creation that many times
    def get_user_input(self):
        num_portfolios = int(input("How many portfolios would you like to create: ")) #how many times to run the loop 
        if num_portfolios == 0: 
            main_menu() #if 0 selected then will go back to the main menu
        else:
            for i in range(num_portfolios):
                self.create_portfolio() #runs create portfolio function as many times as the user wanted to create portfolios


#The creation of portfolios
    def create_portfolio(self):
        portfolio_name = input("Enter the portfolio name: ") #asking for portfolio name, will put in the dictionary
        num_stocks = int(input(f"How many stocks for {portfolio_name}? ")) #need to know how many stocks to add to the dictionary
        if num_stocks == 0:
            main_menu() #go back to main menu if 0 selected
        stocks = [] #set up stocks list which will be added to the dictionary for the respecive portfolio
        for i in range(num_stocks):
            stock = input(f"Enter Stock Option {i}: ") #asks for the stocks for the number of times that they said how many stocks
            stocks.append(stock) #add the stocks onto the stock list

        # Remove leading and trailing whitespaces from each stock symbol
        stocks = [stock.strip() for stock in stocks] #strips the stocks just so spaces do not make it messy
        self.portfolios[portfolio_name] = stocks #now pass the stocks list to the dictionary for the respective portfolio
        print(f"Portfolio '{portfolio_name}' holds stocks: {', '.join(stocks)}") #show what is in the stocks so they can make sure they had done it right, if not they can edit it later


#Selection of portfolios is important for the functions of editing, optimial portfolio creation and stock alert system
    def select_portfolio(self):
        print("\nSelect a portfolio:")
        print("0. Go Back") 
        for i, portfolio_name in enumerate(self.portfolios.keys(), 1): #for the amount of portfolios it prints the portfolio names
            print(f"{i}. {portfolio_name}")

        selected_index = option_checker(0,len(self.portfolios)) #have an option checker that checks the input is within range, for this it is specific to the number of options which is equal to the number of portfolios
        if selected_index==0:
            main_menu() #if 0 is selected then go back to the main menu
            
        else:
            selected_portfolio_name = list(self.portfolios.keys())[selected_index - 1]
            return selected_portfolio_name #if 0 not selected then the choice from the option checker is used to get the selected portfolio name which can be used to select from the portfolios dictionary
    

#We need a way of overwriting a portfolio to avoid having portfolios that are no longer needed, creating clutter
    def edit_portfolio(self, portfolio_name):
        print(f"\nEditing Portfolio '{portfolio_name}':")
        existing_stocks = self.portfolios[portfolio_name] #just a clearer variable for the stocks we have in the selected portfolio

        num_stocks = int(input(f"How many stocks for {portfolio_name}? ")) #similar code to the creation, asking for how many stocks for the loop of asking for the stock
        new_stocks = [] #list for the new stocks that will replace the old stocks in the selected portfolio
        for i in range(num_stocks):
            stock = input(f"Enter Stock Option {i}: ")
            new_stocks.append(stock) #same as creation with the stocks being asked for the number of stocks they said there are, they are added together in the list

        new_stocks = [stock.strip() for stock in new_stocks] #removing any uneeded spaces

        self.portfolios[portfolio_name] = new_stocks #updating the portfolio diction for the selected portfolio with the new stocks, rewriting it

        print(f"Portfolio '{portfolio_name}' updated with new stocks: {', '.join(new_stocks)}") #tells the user what the selected portfolio now holds

    
#Showing the portfolios just gives the user a way of viewing what is in all of the portfolios on one screen
    def show_all_portfolios(self):
        print("\nAll Portfolios:")
        for portfolio_name, stocks in self.portfolios.items(): #print the portfolio name with its corresponding stocks for the amount of portfolios there are
            print(f"{portfolio_name}: {', '.join(stocks)}")



#TRYING TO GET ALL STOCKS IN A LIST
    def portalert(self, portfolio_name):
        stocks = self.portfolios[portfolio_name]
        receiver_email=input("Email:")
        for stock in stocks:
            alert(stock, receiver_email)


#Optimization is the biggest piece of code in the class as it requires many calculations to reach the state where constraints can be passed into the SLSQP algorithm and things like the sharpe ratio can be calculated
    def optimize_portfolio(self, selected_portfolio, risk_free_rate=0.02): #risk free ratio is set to 2 percent but it can flucuate so setting it explcitly may not be the best but it is normally 2-3% 
        money = float(input(f"Budget for Portfolio '{selected_portfolio}': ")) #asking for budget so can output the actual amoutn to put into the stocks not just the percentages
        years_back = int(input(f"How many years back should the analysis consider for {selected_portfolio}? ")) #need stock information to analyse so ask the user what time period they consider important
        
        
        end_date = datetime.today() #stock information will be current
        start_date = end_date - timedelta(days=years_back * 365) #start from when the user specified

        adj_close = pd.DataFrame() #create dataframe for the adjusted close prices that I will get
        for stock in self.portfolios[selected_portfolio]: #for the amoutn of stocks in the selected portfolio
            try:
                data = yf.download(stock, start=start_date, end=end_date) #get the stocks data from the date specified
                adj_close[stock] = data['Adj Close'] #want the adjusted close as this included dividens so is more informative for the user who many benefit from this
            except Exception as e:
                print(f"Error downloading data for {stock}: {e}") #if there is an exception then will give an error essage

        returnlog = np.log(adj_close / adj_close.shift(1)) #calculate the log from the adjusted close prices and the prices from the day before, giving an idea of the trends day to day

        cov = returnlog.cov() * 252 #get the covariance, 252 as removes weekends and holidays in the year

        def standard_deviation(weights, cov):
            variance = weights.T @ cov @ weights #calcuate variance before standard deviation, does the weights transpose multiplied with the covariance matrix and weights
            return np.sqrt(variance) #get the square root of this representing the risk/volatility

        def expected_return(weights, returnlog):
            return np.sum(returnlog.mean() * weights) * 252 #expected return is just the mean of the retrun log function which represented the common trend times the amount of days it will happen, so uses past performance to estimate future

        def sharpe_ratio(weights, returnlog, cov, risk_free_rate): #sharpe ratio is a widely used metrix for how good a portfolio is as it balances the return on investement with the risk
            return (expected_return(weights, returnlog) - risk_free_rate) / standard_deviation(weights, cov) #computes the sharpe ratio equation

        def neg_sharpe_ratio(weights, returnlog, cov, risk_free_rate):
            return -sharpe_ratio(weights, returnlog, cov, risk_free_rate) #did the negative as not sure if SciPy has a maximum but know it has a minimum for sure so went with that

        upper = float(input(f"How big of a percentage are you willing to put into a single stock for {selected_portfolio}? ")) / 100 #the start of the constraints with the amount the user would put into 1 stocks requested
        constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1} #constaints dictionary, equality constraint making the sum of the weights of all stocks is equal to 1 so the capital is fully used
        #the lambda function defines the equality condition taking the weights as input and getting the sum of weights-1, ensuring is equal to 1
        bounds = [(0, upper) for _ in range(len(self.portfolios[selected_portfolio]))]
        initial_weights = np.array([1 / len(self.portfolios[selected_portfolio])] * len(self.portfolios[selected_portfolio])) #intial weights are just set to equal eachother for the stocks by doing 1 divided by the number of stocks

        optimized_results = minimize(neg_sharpe_ratio, initial_weights, args=(returnlog, cov, risk_free_rate),method='SLSQP', constraints=constraints, bounds=bounds) #calculates the best weights given the constraints for the SLSQP algorithm

        optimal_weights = optimized_results.x #create a variable for the best weights
        
        print(f"\nSuggestions are made based of the Sequential Least Squares Programming algorithm, which uses constraints to optimise the sharpe ratio and estimate the best investement portfolio decision")
        print(f"\nSuggested Investment for Portfolio '{selected_portfolio}':")
        for stock, weight in zip(self.portfolios[selected_portfolio], optimal_weights):
            print(f"{stock}: {(weight * money):.2f}") #for each stock in the portfolio, gives the suggested amount to put into it from the optimal weight, need 2 dp

        optimal_portfolio_return = expected_return(optimal_weights, returnlog) #from the calculations before can give the user the expected return, standard deviation for risk and sharpe ratio
        optimal_portfolio_volatility = standard_deviation(optimal_weights, cov)
        optimal_sharpe_ratio = sharpe_ratio(optimal_weights, returnlog, cov, risk_free_rate)

        print(f"\nExpected Annual Return: {optimal_portfolio_return:.2f}") #gives the calculated metrics with small explanation, 2dp given for uniformity
        print("The expected return of the portfolio is the average return you can earn")
        print()
        print(f"Expected Volatility: {optimal_portfolio_volatility:.2f}")
        print("The volatility is an estimate of the future standard deviation giving information on the risk associated with the investments suggested")
        print()
        print(f"Sharpe Ratio: {optimal_sharpe_ratio:.2f}")
        print("The sharpe ratio considers the potential return on investments along with the standard deviations meaning the result is a balance of return and risk")

        plt.figure(figsize=(10, 6)) #now can plot the percentages for visualisatio of where to put the money
        plt.bar(self.portfolios[selected_portfolio], (optimal_weights * 100)) #times 100 for the percentage
        plt.xlabel('Assets')
        plt.ylabel('Optimal Percentage of Budget Allocation')
        plt.title(f'Optimal Portfolio Percentages for {selected_portfolio}')
        plt.show()

        while True:
            print("Enter 1 to go to main menu") #from this page the graph will be outputted and the user can view it but after they are done they will need to either go back to main menu or do another one
            print("Enter 2 to create a new optimal portfolio")
            inputo=input("Selection: ")
            if inputo=='1':
                main_menu()
                break
            elif inputo=='2':
                option4()
                break
            else:
                print("Invalid Input")


#Visualisation of the stocks past performance in the last 2 years
    def plot_portfolio_candlesticks(self, selected_portfolio):
        stocks = self.portfolios[selected_portfolio] #get the stocks from the dictionary for the selected portfolio

        stock_data = {} #initialise dictionary for the stocks and their data
        print("Stock Data for the past 2 years")
        end_date = datetime.today()
        start_date = end_date - timedelta(days=2 * 365)  # 2 years
        for stock in stocks:
            try:
                data = yf.download(stock, start=start_date, end=end_date, interval='1d')
                stock_data[stock] = data #get the stocks data for the past 2 years and then put it into the dictionary for that stocks, do for how many stocks there are in the selected portfolio
            except Exception as e:
                print(f"Error downloading data for stock")

        for stock, data in stock_data.items():
            mpf.plot(data, type='candle', title=stock, ylabel='Price', volume=True, style='yahoo', figscale=1.5) #plot the prices for each stocks in a candlesticks graph
        

        while True: #like before the user can view the graphs bt will then eventually need to go back to the main menu of look at the performance for another portfolio
            print("Enter 1 to go to main menu")
            print("Enter 2 to visualise another portfolio")
            inputo=input("Selection: ")
            if inputo=='1':
                main_menu()
                break
            elif inputo=='2':
                option3()
                break
            else:
                print("Invalid Input")

        
            



## Alert System Functions

This code is run when a portfolio is selceted, the last function is the scheduler that calls the second last function formulating the message. The first 2 functions are used to get the data and determine if there is a pattern seen. To improve the code, the file holding the app password would have been much better if it had some sort of security as at the moment it is visible. It has been left like that for now but I could have either added security or asked the user to create an app password for their email and store their password in the file. Would probably do the latter as not sure how to add security but left how it is for ease of use at the momement. 

The alert system runs but continues to run as expected but was not sure how to come back to the main menu without disrupting the running of the tracking. All other functions work and the alert system works if run straight away but as it is a tracking system so wanted to run it the whole time but didn't know how to run it the whole time but also interact with the system once run. Decided to change it to run straight away with the scheduler code commented out just to make the code still runa dn the email will send on command with an update but this is not what I wanted.

In [19]:
#Get the data for the stock, remeber this is run in a loop from portalert so should be for all stocks in the portfolio
def get_data(stock):
    ticker_symbol = stock #define the symbol
    ticker = yf.Ticker(ticker_symbol) #using the yfinance library to create an object

    return ticker.history(period="2d", interval='1d') #get the recent data


#Testing for engulfing pattern
def test_engulfing(df):#uses open and close for last and previous, remeber close is after open
    last_open = df.iloc[-1, :].Open
    last_close = df.iloc[-1, :].Close
    previous_open = df.iloc[-2, :].Open
    previous_close = df.iloc[-2, :].Close

    if (previous_open < previous_close
            and last_open > previous_close
            and last_close < previous_open):
        return 1  # Bullish Engulfing Pattern as it is rising
    elif (previous_open > previous_close
          and last_open < previous_close
          and last_close > previous_open):
        return 2  # Bearish Engulfing Pattern as it is dropping
    else:
        return 0  # No Engulfing Pattern


def system(portfolio_name): #Need to create this as a function that can be run on selection instead of running the actual alert code as need scheduler
    with open("Email Credentials.txt", "r") as file: #uses the file to get the password, insecure this way, could have asked user to do it
        email_password=file.read().strip()
    #scheduler = BlockingScheduler(job_defaults={'misfire_grace_time': 15*60}) #grace time of 15 mins
    optimizer.portalert(portfolio_name)
    #scheduler.add_job(optimizer.portalert, 'cron', args=[portfolio_name], day_of_week='mon-fri', hour=9, minute=0, timezone='GMT')#runs the alert code at 9am with specified arguement
    #scheduler.start()
    main_menu()



#Message selection and formulation
def alert(stock, receiver_email):
    em=EmailMessage()
    
    msg = "Trading Signal Message\n"  # Start of message
    stan = "No Update As Of Yet. " #default/standard message

    
    historical_data = get_data(stock) #selects which message to create
    if test_engulfing(historical_data) == 2:
        msg = msg + str(stock + ": the signal is bearish") + "\n"
    elif test_engulfing(historical_data) == 1:
        msg = msg + str(stock + ": the signal is bullish") + "\n"
    else:
        msg = msg + stan #standard is no pattern

#have the stockalertsystem2023 email to send
    sender_email="stockalertsystem2023@gmail.com" #where to send the email from
    email_subject = 'Trading Signal Alert'

    with open("Email Credentials.txt", "r") as file:
        email_password=file.read().strip()#just want the password if spaces are there
    
    #have to create elements of the email
    em['From'] = sender_email
    em['To'] = receiver_email
    em['Subject'] = email_subject
    em.set_content(msg)

    context = ssl.create_default_context()

    server = smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) #creates a SMTP object using SSL for encrypted communication using the gmail server
    server.ehlo() #sends an EHLO command which is part of the protocol identifying the client to the server
    server.login(sender_email, email_password) #use the details to login
    server.sendmail(sender_email, receiver_email, em.as_string()) #specify where to send and from with message specified
    server.close() #stop communication with the server





## Menu System Functions

The menu system is just a way to show the user the options they have and a way to call the required functions/classes to perform the task. An option checker is useful to check the inputs of option selections. The way the code works is explained through comments. The menu system is not in a class, just given as functions as a class is not needed with no instances created in the emnu system. To improve on the menu system, there should be more checks to make sure the code does not break which it can on some inputs.

In [20]:
#Checker function will be used everytime a selection of options is needed
def option_checker(min_option, max_option):
    while True:
        user_input = input("Enter an option between {} and {}: ".format(min_option, max_option)) #makes sure user knows the options they have

        try:
            user_input1=int(user_input)
            # Check if the choice is within the specified range
            if min_option <= user_input1 <= max_option:
                print("You Selected Option", user_input1)
                return user_input1 #returns their choice if it fits in the specified range
            else:
                print("Input must be an integer between", min_option, "and", max_option,". You entered", user_input, "Please try again.") #if an integer then just say out of range
        except ValueError:
            print("Invalid input. Please enter a valid option.") #if not an integer then say it is invalid



def main_menu(): #show the options to the user
    print()
    print("---------------Main Menu---------------")
    print("1. Portfolio Creator / Editor")
    print("2. Portfolio Viewer")
    print("3. Portfolio Stock Visualisation")
    print("4. Optimal Portfolio Creator")
    print("5. Alert System Setup")
    print("6. Exit Program")
    user_inputm=option_checker(1, 6)
    #specify the options and the functions run after selection
    if user_inputm==1:
        option1()
    elif user_inputm==2:
        option2()
    elif user_inputm==3:
        option3()
    elif user_inputm==4:
        option4()
    elif user_inputm==5:
        option5()
    elif user_inputm==6:
        option6()
        


#Portfolio creator and editor
def option1():
    print()
    print("---------------Portfolio Creator / Editor---------------")
    print("0. Go Back")
    print("1. Creator")
    print("2. Editor")
    min_option1=0
    max_option1=2
    user_input1=option_checker(min_option1, max_option1) #make sure the user wanted to press on this giving an option to go back

    if user_input1==0:
        main_menu()
    elif user_input1==1:
        optimizer.get_user_input() #runs get user input and portfolio creation code then shows main menu
        main_menu()
    elif user_input1==2:
        selected_portfolio=optimizer.select_portfolio()
        optimizer.edit_portfolio(selected_portfolio) #after selecting which portfolio to edit, then run the edit code
        main_menu() #return back to main menu


#Showing the portfolios
def option2(): 
    optimizer.show_all_portfolios() #option is just to show the portfolios so does this by running that code
    print("Enter 1 to go back to main menu")
    back1 = option_checker(1, 1) #shows the portfolios but need to go back eventually so just have an option of 1 to go back
    while back1 != 1:
        print("Invalid Input, enter 1 for main menu")
        back1 = option_checker(1, 1)

    main_menu()     


#Portfolio stock visualisation
def option3():
    print()
    print("---------------Portfolio Stock Visualisation---------------")
    print("0. Main Menu")
    print("1. Continue")
    sure = option_checker(0, 1) #again give option to go back if didn't mean to press this option
    if sure==0:
        main_menu()
    elif sure==1:
        selected_portfolio=optimizer.select_portfolio()
        optimizer.plot_portfolio_candlesticks(selected_portfolio) #if continued the select which portfolio then plotting graph code is run
    else:
        print("Invalid Input")
        sure=option_checker(0,1) #if 0 or 1 not given then invalid and run again


#Optimal portfolio creator
def option4():
    print()
    print("---------------Optimal Portfolio Creator---------------")
    print("0. Main Menu")
    print("1. Continue")
    sure = option_checker(0, 1) #again make sure wanted to do this option
    if sure==0:
        main_menu()
    elif sure==1:
        selected_portfolio=optimizer.select_portfolio()
        optimizer.optimize_portfolio(selected_portfolio) #select which portfolio and run that portfolio in the optimal code
    else:
        print("Invalid Input")
        sure=option_checker(0,1) #runs again if not a valid input


#Alert system setup
def option5():
    print()
    print("---------------Alert System Setup---------------")
    print("0. Main Menu")
    print("1. Continue")
    sure5=option_checker(0,1) #make sure option wanted
    if sure5==0:
        main_menu()
    elif sure5==1:
        print("You will need to select the portfolio/s seperately to run the alert system")
        print("You will also need to reselect the portfolio on this list if any changes are made")
        #print("Get an app password from google through their 2-step verification and add the password into the Email Credentials.txt file")
        #if password not given how it is now and get user to get their own password then would ask them to get their password and would set sender and reciever email as the same on asking for reciever
        selected_portfolio=optimizer.select_portfolio()
        system(selected_portfolio)
    else:
        print("Invalid Input")
        sure5=option_checker(0,1)

#Exiting the program
def option6():
    print("Exiting program, happy to help") #just a print statement and then nothing so ends the running of the program





optimizer = PortfolioOptimizer()




# Cell to Run the System

In [ ]:

main_menu()


## References

https://www.youtube.com/watch?v=9GA2WlYFeBU, Video by Ryan O'Connell on portfolio optimization in python, the structure of my code (portfolio optimization) was created by following along with his video and process which is similar to others

https://geek.sg/blog/comparison-portfolio-optimization-technique-monte-carlo-slsqp-bayesian, Comparison of 3 different optimisation techniques, used for research into different optimisation algorithms commonly used in finance and their advantages/disadvantages

https://www.kaggle.com/code/vijipai/lesson-7-constrained-portfolio-optimization, Kaggle lession on contrained portfolio optimization, used to understand the flow of creating the optimisation with explanations

https://towardsdatascience.com/portfolio-optimization-with-scipy-aa9c02e6b937, Portfolio optimization with SciPy, used for research into using the SLSQP algorithm with step-by-step code explanations

https://www.portfoliovisualizer.com/optimize-portfolio, Portfolio Visualizer-Optimization, used to see if my optimisation was working with similar results but could also try with different options on the site